<a href="https://colab.research.google.com/github/221ChungJ/FinBERT-Financial-Sentiment-Analyzer/blob/main/FINBERT_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q --upgrade keras-nlp tensorflow
%pip install -q --upgrade keras  # Upgrade to Keras 3.
%pip install -q --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00


In [ ]:
import os


import requests
import json
import nltk
import tensorflow as tf
import keras_nlp
import pandas as pd
import numpy as np
from google.colab import drive
from huggingface_hub import hf_hub_download
from tensorflow import keras
from numpy.random import seed
from keras.models import model_from_json
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.legacy import Adam

drive.mount('/content/drive')
os.environ["KERAS_BACKEND"] = "tensorflow"
policy = keras.mixed_precision.Policy("mixed_float16")
keras.mixed_precision.set_global_policy(policy)
keras.utils.set_random_seed(42)
seed(42)
tf.random.set_seed(42)
print(tf.executing_eagerly())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
True


In [ ]:
# Preprocessing params
PRETRAINING_BATCH_SIZE = 128
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128
MASK_RATE = 0.15
PREDICTIONS_PER_SEQ = 32

# Model Params
NUM_LAYERS = 8
MODEL_DIM = 512
INTERMEDIATE_DIM = 1024
NUM_HEADS = 8
DROPOUT = 0.1
NORM_EPSILON = 1e-12

# Trainging params
PRETRAINING_LEARNING_RATE = 5e-5
PRETAINING_EPOCHS = 50
FINETUNING_LEARNING_RATE_WIKI = 5e-5
FINETUNING_LEARNING_RATE_FINANCE = 7e-5
FINETUNING_EPOCHS = 3

In [ ]:
wiki_train_df_1 = pd.read_parquet('/content/drive/MyDrive/Project Borger/Wikipedia/train-00000-of-00002.parquet')
wiki_train_df_2 = pd.read_parquet('/content/drive/MyDrive/Project Borger/Wikipedia/train-00001-of-00002.parquet')
wiki_val_df = pd.read_parquet('/content/drive/MyDrive/Project Borger/Wikipedia/validation-00000-of-00001.parquet')
wiki_val_df = wiki_val_df['text']

wiki_train_df = pd.concat([wiki_train_df_1, wiki_train_df_2])
wiki_train_df= wiki_train_df['text']


In [ ]:
# Get financial sentiment data
articles_df_1  = pd.read_parquet('/content/drive/MyDrive/Project Borger/financial-news-data/financial_data.parquet.gzip')
articles_df_1 = articles_df_1 ['Article']

articles_df_2 = pd.read_parquet('/content/drive/MyDrive/Project Borger/more-financial-news-data/more_financial_data.paraquet')
articles_df_2 = articles_df_2['0']

articles_df = pd.concat([articles_df_1, articles_df_2])
articles_train_df, articles_test_df = train_test_split(articles_df, test_size=0.2, random_state=42)

articles_train_size = len(articles_train_df)


In [ ]:
# Load SST-2.
sst_train_ds = tf.data.experimental.CsvDataset(
    "/content/drive/MyDrive/Project Borger/SST-2/train.tsv", [tf.string, tf.int32], header=True, field_delim="\t"
).batch(FINETUNING_BATCH_SIZE)
sst_val_ds = tf.data.experimental.CsvDataset(
    "/content/drive/MyDrive/Project Borger/SST-2/dev.tsv", [tf.string, tf.int32], header=True, field_delim="\t"
).batch(FINETUNING_BATCH_SIZE)

print(sst_train_ds.unbatch().batch(1).take(1).get_single_element())


(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'hide new secretions from the parental units '], dtype=object)>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)


In [ ]:
SENTIMENT_MAPPING = {
    'neutral': 2,
    'positive': 1,
    'negative': 0,
}

phraseBank_FiQA_df = pd.read_csv("/content/drive/MyDrive/Project Borger/data.csv")

phraseBank_FiQA_df['sentiment_numerical'] = phraseBank_FiQA_df['Sentiment'].map(SENTIMENT_MAPPING)
phraseBank_FiQA_df.drop('Sentiment', axis=1, inplace=True)

print(phraseBank_FiQA_df.dtypes)

phraseBank_FiQA_texts = phraseBank_FiQA_df['Sentence'].values
phraseBank_FiQA_sentiments = phraseBank_FiQA_df['sentiment_numerical'].values

mapping = {
    'Neutral': 2,
    'Positive': 1,
    'Negative': 0
}

sentiment_annotation_df = pd.read_csv("/content/drive/MyDrive/Project Borger/sentiment_annotated_with_texts.csv", usecols=['text', 'true_sentiment'])

sentiment_annotation_df['true_sentiment_numerical'] = sentiment_annotation_df['true_sentiment'].map(mapping)
sentiment_annotation_df.drop('true_sentiment', axis=1, inplace=True)

sentiment_annotation_texts = sentiment_annotation_df['text'].values
sentiment_annotation_sentiments = sentiment_annotation_df['true_sentiment_numerical'].values

texts = np.concatenate((sentiment_annotation_texts, phraseBank_FiQA_texts))
sentiments = np.concatenate((sentiment_annotation_sentiments, phraseBank_FiQA_sentiments))

finetuning_ds = tf.data.Dataset.from_tensor_slices((texts, sentiments))

dataset_size = sum(1 for _ in finetuning_ds)

train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = int(0.1 * dataset_size)

finetuning_ds = finetuning_ds.shuffle(buffer_size=dataset_size, seed=42)

finetuning_train_ds = finetuning_ds.take(train_size).batch(FINETUNING_BATCH_SIZE)
finetuning_val_test_ds = finetuning_ds.skip(train_size)


finetuning_val_ds = finetuning_val_test_ds.take(val_size).batch(FINETUNING_BATCH_SIZE)
finetuning_test_ds = finetuning_val_test_ds.skip(val_size)



Sentence               object
sentiment_numerical     int64
dtype: object


In [ ]:
# Download vocabulary data.
vocab_file_wiki = keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/keras-nlp/examples/bert/bert_vocab_uncased.txt",
)

vocab_file_finance = "/content/drive/MyDrive/Project Borger/FinVocab-Uncased.txt"

with open(vocab_file_finance, 'r') as f:
    vocab_finance = f.read().splitlines()

with open(vocab_file_wiki, 'r') as f:
    vocab_wiki = f.read().splitlines()

vocab_combined = list(set(vocab_finance + vocab_wiki))

231508/231508 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
wiki_train_ds = tf.data.Dataset.from_tensor_slices(wiki_train_df)

wiki_val_ds = tf.data.Dataset.from_tensor_slices(wiki_val_df)

financial_train_ds = tf.data.Dataset.from_tensor_slices(articles_train_df)

financial_val_ds = tf.data.Dataset.from_tensor_slices(articles_test_df)

print(financial_train_ds.batch(4).take(1).get_single_element())

tf.Tensor(
[b'March 7, 2018 / 8:48 PM / in 4 hours U.S. judge says opioid settlement roadblocks may prompt trials Nate Raymond 3 Min Read (Reuters) - A federal judge pushing for a settlement in lawsuits by state and local governments against drug companies over their roles in the U.S. opioid epidemic said on Wednesday that trials may be needed after both sides identified barriers to reaching a deal. U.S. District Judge Dan Polster in Cleveland in an order said the parties at a hearing on Tuesday to discuss the status of settlement talks indicated that a \xe2\x80\x9climited litigation track\xe2\x80\x9d was needed. Polster said that would include motions and so-called bellwether trials, essentially test cases used in mass litigation in the United States to help both sides gauge the range of damages and define settlement options. Related Coverage Kentucky sues drug distributor AmerisourceBergen over opioid epidemic \xe2\x80\x9cThe parties reported important and substantial progress on sev

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = vocab_combined,
    sequence_length = SEQ_LENGTH,
    lowercase = True,
    strip_accents = True,
)

masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size = tokenizer.vocabulary_size(),
    mask_selection_rate = MASK_RATE,
    mask_selection_length = PREDICTIONS_PER_SEQ,
    mask_token_id = tokenizer.token_to_id("[MASK]"),
)

def preprocess(inputs):
    inputs = tokenizer(inputs)
    outputs = masker(inputs)

    features = {
        "token_ids": outputs["token_ids"],
        "mask_positions": outputs["mask_positions"]
    }
    labels = outputs["mask_ids"]
    weights = outputs["mask_weights"]
    return features, labels, weights

def filter_short_text(text_tensor):
    text_length = tf.strings.length(text_tensor)
    return tf.reduce_any(text_length > 100)

pretrain_wiki_train_ds = (
    wiki_train_ds
    .filter(filter_short_text)
    .map(preprocess, num_parallel_calls = tf.data.AUTOTUNE)
    .batch(PRETRAINING_BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

pretrain_wiki_val_ds = (
    wiki_val_ds
    .filter(filter_short_text)
    .map(preprocess, num_parallel_calls = tf.data.AUTOTUNE)
    .batch(PRETRAINING_BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

pretrain_financial_train_ds = (
    financial_train_ds
    .map(preprocess, num_parallel_calls = tf.data.AUTOTUNE)
    .batch(PRETRAINING_BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

pretrain_financial_val_ds = (
    financial_val_ds
    .map(preprocess, num_parallel_calls = tf.data.AUTOTUNE)
    .batch(PRETRAINING_BATCH_SIZE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

wiki_train_size = sum(1 for _ in pretrain_wiki_train_ds) * PRETRAINING_BATCH_SIZE
wiki_val_size = sum(1 for _ in pretrain_wiki_val_ds) * PRETRAINING_BATCH_SIZE

print(wiki_train_size)
print(wiki_val_size)



KeyboardInterrupt: 

In [ ]:
inputs = keras.Input(shape=(SEQ_LENGTH,), dtype="int32")

embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size = tokenizer.vocabulary_size(),
    sequence_length = SEQ_LENGTH,
    embedding_dim = MODEL_DIM,
)
outputs = embedding_layer(inputs)

outputs = keras.layers.LayerNormalization(epsilon=NORM_EPSILON)(outputs)
outputs = keras.layers.Dropout(rate=DROPOUT)(outputs)

for i in range(NUM_HEADS):
    outputs = keras_nlp.layers.TransformerEncoder(
        intermediate_dim = INTERMEDIATE_DIM,
        num_heads = NUM_HEADS,
        dropout = DROPOUT,
        layer_norm_epsilon = NORM_EPSILON,
    )(outputs)

encoder_model = keras.Model(inputs, outputs)
encoder_model.summary()
print(encoder_model.output_shape)

NameError: name 'tokenizer' is not defined

In [ ]:
inputs = {
    "token_ids": keras.Input(shape=(SEQ_LENGTH,), dtype="int32", name="token_ids"),
    "mask_positions": keras.Input(
        shape = (PREDICTIONS_PER_SEQ,), dtype="int32", name="mask_positions"
    )
}

encoded_tokens = encoder_model(inputs["token_ids"])

outputs = keras_nlp.layers.MaskedLMHead(
    token_embedding = embedding_layer.token_embedding,
    activation = "softmax",
)(encoded_tokens, mask_positions=inputs["mask_positions"])

pretraining_model = keras.Model(inputs, outputs)
pretraining_model.compile(
    loss = "sparse_categorical_crossentropy",
    optimizer = keras.optimizers.AdamW(PRETRAINING_LEARNING_RATE),
    weighted_metrics = ["sparse_categorical_crossentropy"],
    jit_compile = True,
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

pretraining_model.fit(
    pretrain_wiki_train_ds,
    validation_data = pretrain_wiki_val_ds,
    epochs = 50,
    shuffle=True,
    callbacks=[early_stopping]
)

pretraining_model.fit(
    pretrain_financial_train_ds,
    validation_data=pretrain_financial_val_ds,
    epochs=50,
    shuffle=True,
    callbacks=[early_stopping]
)

encoder_model.save("encoder_model.keras")

NameError: name 'embedding_layer' is not defined

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = vocab_combined,
    sequence_length = SEQ_LENGTH,
    lowercase = True,
    strip_accents = True,
)

@tf.function
def preprocess(sentences, labels):
    tokenized_sentences = tokenizer(sentences)
    # tokenized_sentences = tf.keras.preprocessing.sequence.pad_sequences(tokenized_sentences, maxlen=32)
    return tokenized_sentences, labels

wiki_finetuning_train_ds = (
    sst_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
).prefetch(tf.data.AUTOTUNE)

wiki_finetuning_val_ds = (
    sst_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
).prefetch(tf.data.AUTOTUNE)

finance_finetuning_train_ds = (
    finetuning_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

finance_finetuning_val_ds = (
    finetuning_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

finance_finetuning_test_ds = (
    finetuning_test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
encoder_model = keras.models.load_model("/content/drive/MyDrive/Project Borger/encoder_model.keras", compile=False)

print(encoder_model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, 128, 512)            │      25,100,800 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization                  │ (None, 128, 512)            │           1,024 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_1                │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_2                │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_3                │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_4                │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_5                │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_6                │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_7                │ (None, 128, 512)            │       2,102,784 │
│ (TransformerEncoder)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,924,096 (159.93 MB)

 Trainable params: 41,924,096 (159.93 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
encoder_model = keras.models.load_model("/content/drive/MyDrive/Project Borger/encoder_model.keras", compile=False)

inputs = keras.Input(shape=(SEQ_LENGTH,), dtype="int32")

# Encoder model
encoded_tokens = encoder_model(inputs)

pooled_tokens = keras.layers.GlobalAveragePooling1D()(encoded_tokens)

outputs = keras.layers.Dense(3, activation="softmax")(pooled_tokens)

# Finetuning model
finetuning_model = keras.Model(inputs, outputs)


finetuning_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=FINETUNING_LEARNING_RATE_WIKI),
    metrics=["sparse_categorical_accuracy"]
)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

print(finetuning_model.summary())

# Fine-tune the model
finetuning_model.fit(
    wiki_finetuning_train_ds,
    validation_data=wiki_finetuning_val_ds,
    epochs=3,
)

finetuning_model.fit(
    finance_finetuning_train_ds,
    validation_data=finance_finetuning_val_ds,
    epochs=10,
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional (Functional)              │ (None, 128, 512)            │      41,924,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,925,635 (159.93 MB)

 Trainable params: 41,925,635 (159.93 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
   2105/Unknown 163s 42ms/step - loss: 0.5598 - sparse_categorical_accuracy: 0.6883

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2105/2105 ━━━━━━━━━━━━━━━━━━━━ 188s 54ms/step - loss: 0.5597 - sparse_categorical_accuracy: 0.6883 - val_loss: 0.4636 - val_sparse_categorical_accuracy: 0.8073
Epoch 2/3
2105/2105 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - loss: 0.2343 - sparse_categorical_accuracy: 0.9076 - val_loss: 0.4986 - val_sparse_categorical_accuracy: 0.8108
Epoch 3/3
2105/2105 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - loss: 0.1579 - sparse_categorical_accuracy: 0.9417 - val_loss: 0.5676 - val_sparse_categorical_accuracy: 0.8188
Epoch 1/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 121s 352ms/step - loss: 1.8121 - sparse_categorical_accuracy: 0.4315 - val_loss: 0.8831 - val_sparse_categorical_accuracy: 0.5498
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.8401 - sparse_categorical_accuracy: 0.5975 - val_loss: 0.6317 - val_sparse_categorical_accuracy: 0.7097
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.6418 - sparse_categorical_accuracy: 0.7082 - val_loss: 0.4709 - val_sparse_categorical_accuracy: 0.787

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = vocab_combined,
    sequence_length = SEQ_LENGTH,
    lowercase = True,
    strip_accents = True,
)

inputs = keras.Input(shape=(), dtype=tf.string)
tokenized_inputs = tokenizer(inputs)
outputs = finetuning_model(tokenized_inputs)
final_model = keras.Model(inputs, outputs)

final_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=FINETUNING_LEARNING_RATE_WIKI),
    metrics=["sparse_categorical_accuracy"],
    jit_compile=False
)

final_model.save("final_model.keras")
final_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None)                      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ word_piece_tokenizer_1               │ (None, 128)                 │               0 │
│ (WordPieceTokenizer)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional (Functional)              │ (None, 3)                   │      41,925,635 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,925,635 (159.93 MB)

 Trainable params: 41,925,635 (159.93 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
restored_model = tf.keras.models.load_model('final_model.keras')
print(restored_model.summary())
inference_data = tf.constant(["Yeah, if you look at really over the last three quarters, the gross margin decline we've had is really primarily been driven by the mix impact of our recent international M&A, primarily the Pioneer acquisition. And we expect that to continue into Q2, that's the last quarter before we finally lap out of that."])
tokenized_inputs = tokenizer(inference_data)

predictions = restored_model.predict(inference_data)

SENTIMENT_MAPPING = {2: 'neutral', 1: 'positive', 0: 'negative'}
predicted_sentiment = SENTIMENT_MAPPING[tf.argmax(predictions, axis=1).numpy()[0]]
print(predicted_sentiment)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 274 variables whereas the saved optimizer has 4 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 270 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None)                      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ word_piece_tokenizer_1               │ (None, 128)                 │               0 │
│ (WordPieceTokenizer)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional (Functional)              │ (None, 3)                   │      41,925,635 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 125,776,911 (479.80 MB)

 Trainable params: 41,925,635 (159.93 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 83,851,276 (319.87 MB)

None
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
positive


In [ ]:
restored_model = tf.keras.models.load_model("/content/drive/MyDrive/Project Borger/final_model.keras", compile=True)
test_loss, test_accuracy = restored_model.evaluate(finance_finetuning_test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - loss: 2.6982 - sparse_categorical_accuracy: 0.4121
Test Loss: 2.6681323051452637
Test Accuracy: 0.4176904261112213


In [ ]:
SENTIMENT_MAPPING = {2: 'neutral', 1: 'positive', 0: 'negative'}

total = 0
correct = 0
for text, label in finetuning_test_ds:  # Ignore labels if you only want predictions
    prediction = restored_model.predict(tf.constant([text.numpy().decode('utf-8')]))
    predicted_sentiment = SENTIMENT_MAPPING[tf.argmax(prediction, axis=1).numpy()[0]]
    # print("Prediction:", predicted_sentiment)
    # print("Actual:", SENTIMENT_MAPPING[label.numpy()])
    if predicted_sentiment == SENTIMENT_MAPPING[label.numpy()]:
        correct += 1
    total += 1

accuracy = correct / total
print("Accuracy:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━